<h1><center>Análise de Dados</center></h1>
<h3><center>Criar uma base, com pipeline para as outras ias</center></h3>

***

Conteúdo deste notebook:
1. Imports
1. Conectando com a base
1. Separação entre resposta e atributos
1. Definindo o modelo e a pipeline incluindo o SMOTE
1. Definindo o grid de hiperparâmetros
1. Definindo Validação Cruzada e GridSearch
1. Testar Diferentes Divisões de Treino/Teste e Ajustar a Pipeline
1. Vendo os melhores parametro

#### Imports

In [39]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
import numpy as np

#### Conectando com a base

In [40]:
df = pd.read_excel('./base/incluses_tratado.xlsx')

df['Finalidade do Uso do App Incluses'] = df['Finalidade do Uso do App Incluses'].replace(
    {"Encontrar oportunidades de emprego": "sim", 
     "Fazer cursos de qualificação": "sim", 
     "Divulgar conteúdos profissionais": "sim",
     'Divulgar conteúdos profissionais\xa0':"sim" ,
     "Todos acima": "sim", "Não usaria o app": "não"}
     )

display(df.head(5))

,Participante da Comunidade LGBTQIA+,Faixa Etária,Identidade de Gênero,Orientação Sexual,Cidade/Estado,Escolaridade,Usa Apps para Oportunidades de Emprego?,Preferência de Cursos,Desafios de Emprego por Gênero,Interesse em Empreender,Situação no Mercado de Trabalho,Usa Redes Sociais?,Finalidade do Uso do App Incluses
0,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Sim,Prefiro cursos on-line,Sim,Não,Empregado (a),Sim,sim
1,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Tenho interesse,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,não
2,Sim,Menos de 18 anos,Mulher cisgênero,Pansexual,Região Norte,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Não,Talvez,Empregado (a),Sim,sim
3,Sim,Menos de 18 anos,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,não
4,Sim,Menos de 18 anos,Homem transgênero,Prefiro não informar,Região Sudeste,Nenhum,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Sim,Desempregado (a),Sim,sim


#### Separação entre resposta e atributos

In [41]:
df_resposta = df['Finalidade do Uso do App Incluses']
df_atributo = df.iloc[:, :-1]

#### Transformando colunas quantitativas em qualitativas

In [42]:
df_resposta = df['Finalidade do Uso do App Incluses']
df_atributo = df.iloc[:, :-1]

#Resposta
label_encoder = LabelEncoder()

#Treino
preprocessador = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool'])),
    ],
    remainder='passthrough',
    verbose_feature_names_out= False 
)

# Aplicando o preprocessador nos atributos
df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())

#### Definindo o modelo e a pipeline incluindo o SMOTE

In [43]:
# Definir o modelo base
model = RandomForestClassifier(random_state=42)

# Definir a pipeline incluindo o SMOTE e o modelo
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', model)
])

#### Definindo o grid de hiperparâmetros

In [44]:
# Definindo os classificadores

classifiers = {
    'knn': KNeighborsClassifier(),
    'naive_bayes': GaussianNB(),
    'decision_tree': DecisionTreeClassifier()
}

# Definindo o grid de hiperparâmetros para problemas multiclasse
param_grid = [
    {  # Parâmetros para KNeighborsClassifier
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': np.arange(1, 10, 1),
        'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'classifier__leaf_size': np.arange(10, 50, 10),
        'classifier__p': [1, 2],  # 1 para Manhattan (L1), 2 para Euclidean (L2)
        'classifier__weights': ['uniform', 'distance'],
        'smote__sampling_strategy': ['auto'],
        'smote__k_neighbors': np.arange(1, 6, 1)
    },
    {  # Parâmetros para GaussianNB
        'classifier': [GaussianNB()],
        'classifier__var_smoothing': np.logspace(0, -9, num=100),  # Param específico do GaussianNB
        'smote__sampling_strategy': ['auto'],
        'smote__k_neighbors': np.arange(1, 6, 1)
    },
    {  # Parâmetros para DecisionTreeClassifier
        'classifier': [DecisionTreeClassifier()],
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__splitter': ['best', 'random'],
        'classifier__max_depth': [None, 2, 4, 6, 8, 10, 12],
        'classifier__min_samples_split': [2, 5, 10, None],
        'classifier__min_samples_leaf': [1, 2, 5, 10, None],
        'classifier__max_features': [None, 'sqrt', 'log2'],
        'classifier__class_weight': [None, 'balanced'],  # Pode ser útil em dados desbalanceados
        'smote__sampling_strategy': ['auto'],
        'smote__k_neighbors': np.arange(1, 6, 1)
    }
]

#### Definindo Validação Cruzada e GridSearch

In [45]:
# Validação cruzada estratificada
cv = StratifiedKFold(n_splits=8)

# Definindo o GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)

#### Testando diferentes divisões de Treino/Teste e ajustar a pipeline:

In [46]:
test_sizes = [0.2, 0.25, 0.3]

# Variáveis para armazenar os melhores resultados para cada classificador
best_results = {
    'KNeighborsClassifier': {'best_test_size': None, 'best_score': -float('inf'), 'best_parameters': None},
    'GaussianNB': {'best_test_size': None, 'best_score': -float('inf'), 'best_parameters': None},
    'DecisionTreeClassifier': {'best_test_size': None, 'best_score': -float('inf'), 'best_parameters': None}
}

# Testar diferentes divisões treino/teste
for test_size in test_sizes:
    # Dividir o conjunto de dados
    X_train, X_test, y_train, y_test = train_test_split(df_atributo, df_resposta, test_size=test_size, random_state=42, stratify=df_resposta)
    
    # Ajustar o GridSearchCV
    grid_search.fit(X_train, y_train)
    
    # Prever no conjunto de teste
    y_pred = grid_search.predict(X_test)
    
    # Calcular a acurácia
    score = accuracy_score(y_test, y_pred)
    
    # Identificar qual classificador está sendo usado e salvar os melhores resultados
    classifier_name = grid_search.best_estimator_.named_steps['classifier'].__class__.__name__
    
    # Mostrar os resultados de cada iteração
    print(f"Modelo: {classifier_name}, Test Size: {test_size}, Score: {score}")
    
    # Verificar se a acurácia é melhor que a melhor anterior
    if score > best_results[classifier_name]['best_score']:
        best_results[classifier_name]['best_score'] = score
        best_results[classifier_name]['best_test_size'] = test_size
        best_results[classifier_name]['best_parameters'] = grid_search.best_params_


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=8.
  warnings.warn(
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
88756 fits failed out of a total of 161440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_trai

Modelo: DecisionTreeClassifier, Test Size: 0.2, Score: 0.9


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
107600 fits failed out of a total of 161440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16144 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^

Modelo: DecisionTreeClassifier, Test Size: 0.25, Score: 0.8333333333333334


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
107600 fits failed out of a total of 161440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16144 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^

Modelo: DecisionTreeClassifier, Test Size: 0.3, Score: 0.8


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [0.78125 0.78125     nan ...     nan     nan     nan]
  warnings.warn(


#### Vendo os melhores parametro

In [48]:
# Exibir os melhores resultados por modelo
for model_name, result in best_results.items():
    print(f"\nModelo: {model_name}")
    print(f"Melhor divisão treino/teste: {result['best_test_size']}")
    print(f"Melhor acurácia: {result['best_score']:.4f}")  # Mostrando a acurácia com 4 casas decimais
    print(f"Melhores parâmetros: {result['best_parameters']}")



Modelo: KNeighborsClassifier
Melhor divisão treino/teste: None
Melhor acurácia: -inf
Melhores parâmetros: None

Modelo: GaussianNB
Melhor divisão treino/teste: None
Melhor acurácia: -inf
Melhores parâmetros: None

Modelo: DecisionTreeClassifier
Melhor divisão treino/teste: 0.2
Melhor acurácia: 0.9000
Melhores parâmetros: {'classifier': DecisionTreeClassifier(), 'classifier__class_weight': None, 'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 5, 'classifier__min_samples_split': 10, 'classifier__splitter': 'best', 'smote__k_neighbors': np.int64(3), 'smote__sampling_strategy': 'auto'}
